## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [1]:
import sys,json
import glob, os
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/")

sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") 
from spinalcordtoolbox.utils.sys import run_proc

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.11, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


## <font color=#B2D732> <span style="background-color: #4424D6"> A/ Initialization

 ### <font color=#4424D6> I. Configuration & parameters </font >

In [2]:
with open('../config/config_CL.json') as config_file:
    config = json.load(config_file) # load config file
signal='raw'
seed_names=['spinalcord_C1','spinalcord_C2','spinalcord_C3','spinalcord_C4','spinalcord_C5','spinalcord_C6','spinalcord_C7','spinalcord_C8','spinalcord_C9'] # define seed names ex: 'spinalcord_ICA-C4'
seed_names=['spinalcord_GM-L','spinalcord_GM-V','spinalcord_GM-D']#,'spinalcord_C1C8-LD','spinalcord_C1C8-LV','spinalcord_C1C8-RD','spinalcord_C1C8-RV',]

target_name=['brain_mask'] # define structure target


### <font color=#4424D6> II. Select files </font>

In [3]:
# One target per subject
data_target=[];ts_target_txt=[];ts_seed_txt ={};data_seed=[];mask_seed={}

for subject_name in config['list_subjects']:
    data_target.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+target_name[0].split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+target_name[0]+'/timeseries/sub_' + subject_name + '_mask_' + target_name[0] + '_timeseries')
    data_seed.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+seed_names[0].split('_')[0]+'/*' + config["coreg_tag"][seed_names[0].split('_')[0]] +'*')[0])
for seed_name in seed_names:
    ts_seed_txt[seed_name]=[]
    for subject_name in config['list_subjects']:
        ts_seed_txt[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/sub_' + subject_name + '_mask_' + seed_name.split('_')[-1] + '_timeseries')

for seed_name in seed_names:
    #mask_seed[seed_name]=glob.glob(config["main_dir"] + config["data"]["ICA"]["spinalcord_dir"]+ 'K_9/comp_rois/' +  '*' + seed_name.split('_')[1] + '*')[0]
    #mask_seed[seed_name]=config["main_dir"] + '/templates/MNI/rois/L-BA4.nii.gz'#    
    mask_seed[seed_name]=glob.glob(config["main_dir"] + config["seed2vox_dir"] + '/masks/'+ seed_name + '.nii.gz')[0]
    
    mask_target=glob.glob(config["main_dir"] + config["masks"][target_name[0].split('_')[0]])[0]

# create output directory if needed
for seed_name in seed_names:
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name)
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/') # folder to store timeseries extraction
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/brain_fc_maps/') # folder to store maps of FC

    

### <font color=#4424D6> III. Initialize class based on this </font>

In [4]:
for seed_name in seed_names:
    seed2voxels=Seed2voxels(config,signal,seed_name,target_name) # initialize the function

## <font color=#B2D732> <span style="background-color: #4424D6"> B/ Data extraction </font></span>
### <font color=#4424D6> I. Time series extraction - Target </font>


In [5]:
target_timeseries,target_timeseries_mean,target_timeseries_pc1=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                                                                run="load",n_jobs=8,smoothing_fwhm=[6,6,6]) # run the analyse for target voxels


### <font color=#4424D6> II. Time series extraction - Seed </font>

In [6]:
seed_timeseries={};seed_timeseries_mean={};seed_timeseries_pc1={}
for seed_name in seed_names[0:4]:
    seed_timeseries[seed_name],seed_timeseries_mean[seed_name],seed_timeseries_pc1[seed_name]=seed2voxels.extract_data(img=data_seed, mask= mask_seed[seed_name], timeseries_txt=ts_seed_txt[seed_name],
                                                                run="extract",n_jobs=8) # run the analyse for target voxels

## <font color=#B2D732> <span style="background-color: #4424D6"> C/ Correlation analysis

In [7]:
output_img={};

for seed_name in seed_names[0:4]:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name.split('_')[-1] + '_s'

    seed2voxels.correlation_maps(seed_timeseries_mean[seed_name],target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,partial=False,n_jobs=8)

## <font color=#B2D732> <span style="background-color: #4424D6"> D/ Statistics
#### <font color=#4424D6> One sample t-test

In [10]:
seed_name=seed_names[2]
ouput_dir='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/2_second_level/' + seed_name +'/'
if not os.path.exists(ouput_dir):
    os.mkdir(ouput_dir)
    run_proc('randomise -i {} -o {} -1 -v 5 -n 1000  -T'.format(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name.split('_')[-1] + '_s_zcorr.nii',
                                                           ouput_dir + seed_name ,
                                                           mask_target))

randomise -i /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-D/brain_fc_maps/21subjects_seed_GM-D_s_zcorr.nii -o /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/2_second_level/spinalcord_GM-D/spinalcord_GM-D -1 -v 5 -n 1000  -T # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook


#### <font color=#4424D6> Two sample t-test

In [12]:
import shutil
#1. select the analyses to compared
seed1='spinalcord_GM-V'
seed2='spinalcord_GM-D'
output_dir=config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/'+seed1+'-'+seed2.split('-')[-1]
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
#2. merge the images from the 2 contrasts in one 4D image
    run_proc('fslmerge -t {} {} {}'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii', 
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed1 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed1.split('_')[-1]+'_s_zcorr',
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed2 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed2.split('_')[-1]+'_s_zcorr'))

#3. Run randomize
    run_proc('randomise -i {} -o {} -d {} -t {} -e {} -m {} -v 5 -n 500  -T'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii',
                                                                                    output_dir + '/'+seed1+'-'+seed2.split('-')[-1],
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.mat',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.con',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.grp',
                                                                                mask_target))

fslmerge -t /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//2_second_level/spinalcord_GM-V-D/spinalcord_GM-V-D_images.nii /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-V/brain_fc_maps/21subjects_seed_GM-V_s_zcorr /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-D/brain_fc_maps/21subjects_seed_GM-D_s_zcorr # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook
randomise -i /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//2_second_level/spinalcord_GM-V-D/spinalcord_GM-V-D_images.nii -o /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//2_second_level/spinalcord_GM-V-D/spinalcord_GM-V-D -d /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//2_second_level/fsl_designs/2paired-t-test/d

In [98]:
input='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_C1C8-L.nii.gz'
mask='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-cord.nii.gz'
output='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-L.nii.gz'
run_proc('fslmaths {} -mas {} {}'.format(input,mask,output))

fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_C1C8-L.nii.gz -mas /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-cord.nii.gz /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-L.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook


(0, '')